<a href="https://colab.research.google.com/github/kawawa123/classifier/blob/master/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
#scikit-learnのライブラリ

In [0]:
#aaa

In [0]:
#可視化ツール
#「pandas_profiling」はバージョンの都合で現在利用できない
#pivottablejs


In [0]:
#http://tekenuko.hatenablog.com/entry/2017/10/13/225354
#https://qiita.com/Mt_Taka/items/e12694aa5a7ed9539216
#https://qiita.com/shin_mura/items/8f1aa1ec90fa4ad6253e
#catboost

In [0]:
#「google colab」と「google drive」を接続
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#「kaggle」にアクセスできるようにする
#https://www.dragonarrow.work/articles/216
import os
import json
f = open("/content/drive/My Drive/kaggle/kaggle.json", 'r')

json_data = json.load(f) #JSON形式で読み込む
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']
# print("KAGGLE_USERNAME: " + os.environ["KAGGLE_USERNAME"])
# print("KAGGLE_KEY: " + os.environ["KAGGLE_KEY"])

In [0]:
!kaggle competitions download -c titanic

In [0]:
import pandas as pd
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [0]:
#何度も利用する時のやり方
'''
!more kaggle.json
# kaggle.jsonをアップロードする

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json

# kaggle APIキーを打ち込む（コンペごとに異なる）
!kaggle competitions download -c titanic

# ディレクトリinputを作成
!mkdir input

# ファイルの移動
!mv train.csv input
!mv test.csv input
'''

In [0]:
#一番最初は必要なライブラリのインストール
!pip install optuna
!pip install heamy
!pip install catboost

In [0]:
!git clone https://github.com/h2oai/pystacknet
!pip install "/content/pystacknet"


'''
!git clone https://gitlab.com/YannBerthelot/kaggle_pystacknet.git
print(os.listdir("kaggle_pystacknet/pystacknet"))
!pip install "kaggle_pystacknet/pystacknet"
import pystacknet
'''
#「pystacknet」はこちらに変えるかもしれない
#参考ＵＲＬ
#https://www.kaggle.com/yannberthelot/pystacknet-working-implementation

In [0]:
#環境設定
import pandas as pd
import numpy as np
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_validate

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
#多分類モデルに必要

In [0]:
#「iris」データ
iris = load_iris()
X = iris.data
y = iris.target
#ここの「y」の数で、「カテゴリ分析」かどうかの判断ができると思う
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#train,validate,testの三つに分けてもいいかも


In [0]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

train_df.isnull().sum()
#列ごとに「null」の数をカウントできる
train_df.fillna(-999,inplace=True)
test_df.fillna(-999,inplace=True)
#nullを一括で「-999」に変更する

x=train_df.drop('Survived',axis=1)
y=train_df.Survived
#今回は「survived」を予測するモデル

x.dtypes
#変数のタイプを調べる
cfi = np.where(x.dtypes != float)[0]
#［categorical_features_indices］
#カテゴリカル変数として扱う要素


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [0]:
cv=5
#交差検証の等分数


In [0]:
#https://github.com/optuna/optuna/issues/1001
#「early stop」に関する参考ＵＲＬ
OPTUNA_EARLY_STOPING = 10

class EarlyStoppingExceeded(optuna.exceptions.OptunaError):
    early_stop = OPTUNA_EARLY_STOPING
    early_stop_count = 0
    best_score = None

def early_stopping_opt(study, trial):
    if EarlyStoppingExceeded.best_score == None:
      EarlyStoppingExceeded.best_score = study.best_value

    if study.best_value < EarlyStoppingExceeded.best_score:
        EarlyStoppingExceeded.best_score = study.best_value
        EarlyStoppingExceeded.early_stop_count = 0
    else:
      if EarlyStoppingExceeded.early_stop_count > EarlyStoppingExceeded.early_stop:
            EarlyStoppingExceeded.early_stop_count = 0
            best_score = None
            raise EarlyStoppingExceeded()
      else:
            EarlyStoppingExceeded.early_stop_count=EarlyStoppingExceeded.early_stop_count+1
    #print(f'EarlyStop counter: {EarlyStoppingExceeded.early_stop_count}, Best score: {study.best_value} and {EarlyStoppingExceeded.best_score}')
    return


# クラス分けモデル
1. RandomForestClassifier
2. ExtraTreesClassifier
3. XGBT
4. LGBM

## optuna
- 選択  
> # trial.suggest_categorical   
>>
>ex. ) 活性化関数にreluとsigmoidのどちらかを選択するとき  
`activation = trial.suggest_categorical('activation', ['relu', 'sigmoid'])`
-  数値の自動調整
> 1. trial.suggest_discrete_uniform(a, b, c)  
> 区間[a, b]内を刻み幅cで区切り、最適値を探します。
>>
> 2. trial.suggest_uniform(a, b)  
> 区間[a, b]内で最適値を探します。  
ex. ) Dropout率を0から1の間で調整するとき  
`dropout_rate = trial.suggest_uniform('dropout_rate', 0, 1)`
> 3. trial.suggest_int  
> int型での調整  
ex. ) 隠れ層のユニット数を[100, 101, ... 499, 500]のどれかから選ぶとき  
`mid_units = int(trial.suggest_discrete_uniform('mid_units', 100, 500, 1))`  
`mid_units = trial.suggest_int('mid_units', 100, 500)`
-  層の数を増やしたい
> # forを利用した計算  
ex. ) for文を使いユニット数100、活性化関数reluの層を増やすとき  
`n_layer = trial.suggest_int('n_layer', 1, 3)`  
`for i in range(n_layer):`  
`    model.add(Dense('100', activation='relu'))`








In [0]:

/content/drive/My Drive/Colab Notebooks/ml/classifier

In [0]:
import os 
os.getcwd()

'C:\\Users\\yuyak\\Google ドライブ\\Colab Notebooks\\ml\\classifier\\script'

In [0]:


print(2+2)

In [0]:
with open('drive/My Drive/python_lib/ori.py',encoding='utf-8') as f:
  code = f.read()
  exec(code)

#必要なライブラリをすべてimportする

In [0]:
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    
    param_grid_rfc = {
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        'min_samples_split': trial.suggest_int("min_samples_split", 7, 15),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        #'max_features': trial.suggest_int("max_features", 3, 10),
        'max_features': 'auto',
        "random_state": 0
    }

    model = RandomForestClassifier(**param_grid_rfc)
    
    # 5-Fold CV / Accuracy でモデルを評価する
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X_train, y=y_train, cv=kf)
    # 最小化なので 1.0 からスコアを引く
    return scores['test_score'].mean()

study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=50,timeout=60, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')


train_rf=study.best_value
rfc_best_param = study.best_params



In [0]:
from sklearn.ensemble import ExtraTreesClassifier

def objective(trial):
    
    param_grid_etc = {
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        'min_samples_split': trial.suggest_int("min_samples_split", 7, 15),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        #'max_features': trial.suggest_int("max_features", 3, 10),
        'max_features': 'auto',
        "random_state": 0
    }

    model = ExtraTreesClassifier(**param_grid_etc)
    
    # 5-Fold CV / Accuracy でモデルを評価する
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X_train, y=y_train, cv=kf)
    # 最小化なので 1.0 からスコアを引く
    return scores['test_score'].mean()

study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=50,timeout=60, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')


train_et=study.best_value
etc_best_param = study.best_params



In [0]:
from xgboost import XGBClassifier
def objective(trial):
    
    param_grid_xgb = {
        'min_child_weight': trial.suggest_int("min_child_weight", 1, 5),
        'gamma': trial.suggest_discrete_uniform("gamma", 0.1, 1.0, 0.1),
        'subsample': trial.suggest_discrete_uniform("subsample", 0.5, 1.0, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform("colsample_bytree", 0.5, 1.0, 0.1),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
        "random_state": 0
    }

    model = XGBClassifier(**param_grid_xgb)
    
    # 5-Fold CV / Accuracy でモデルを評価する
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X_train, y=y_train, cv=kf)
    # 最小化なので 1.0 からスコアを引く
    return scores['test_score'].mean()




study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=50,timeout=60, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

train_xg=study.best_value
xgb_best_param = study.best_params

In [0]:
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
#パラメーターのチューニング情報
from lightgbm import LGBMClassifier

def objective(trial):
    
    param_grid_lgb = {
        'num_leaves': trial.suggest_int("num_leaves", 3, 10),                      ##大規模データの時は「70～80」
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1.0),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 500)    #大規模データの時は「数百～数千」
        "random_state": 0
    }

    model = LGBMClassifier(**param_grid_lgb)
    
    # 5-Fold CV / Accuracy でモデルを評価する
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X_train, y=y_train, cv=kf)
    # 最小化なので 1.0 からスコアを引く
    return scores['test_score'].mean()


study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=50,timeout=60, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

print(study.best_params)
print(study.best_value)
train_lg=study.best_value
lgb_best_param = study.best_params

In [0]:
from sklearn.model_selection import train_test_split
from catboost import Pool
import sklearn.metrics

train_pool = Pool(X_train, y_train, cat_features=cfi)
test_pool = Pool(X_test, y_test, cat_features=cfi)

def objective(trial):
    # トレーニングデータとテストデータを分割
    
    '''
    train_pool = Pool(X_train, y_train, cat_features=cfi)
    test_pool = Pool(X_test, y_test, cat_features=cfi)
    '''
    # パラメータの指定
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50)
    }

    # 学習
    model = CatBoostClassifier(**params)
    model.fit(train_pool)
    # 予測
    preds = model.predict(test_pool)
    pred_labels = np.rint(preds)
    # 精度の計算
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return 1.0 - accuracy

In [0]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)
print(study.best_trial)



In [0]:
model = CatBoostClassifier(**study.best_params)
model.fit(train_pool)
# 予測
preds = model.predict(test_pool)
pred_labels = np.rint(preds)


In [0]:

accuracy_score(preds,y_test.values)

In [0]:
#分析モデルが変更されたときは
#スタッキングの一番最初の変数、「models」も変更する必要がある



train_result=[train_rf,train_et,train_xg,train_lg]
#一応、「train」データの結果をまとめておく


estimators = [
    ('rfc', RandomForestClassifier(**rfc_best_param)),
    ('etc', ExtraTreesClassifier(**etc_best_param)),
    
    ('xgb', XGBClassifier(**xgb_best_param)),
    ('lgb', LGBMClassifier(**lgb_best_param))
]
#分析する個別モデル
model_names=['rf','et','xg','lg']


# 全てのデータのテスト結果を計算

In [0]:
#先に個別モデル
#「estimators」は「クラス分けモデル」の最後に作った変数
RESULT=[]
for ii in range(len(estimators)):
  model=estimators[ii][1]
  model.fit(X_train, y_train)
  y_pred = model.predict_proba(X_test)
  #y_pred = lgb_best.predict_proba(X_test)
  y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

  # 精度 (Accuracy) を計算する
  accuracy = sum(y_test == y_pred_max) / len(y_test)
  RESULT.append(accuracy)
  


# スタッキング
1. Single Stacking
2. Weight Average
3. Multiple Stacking
4. stacknet

- 一般的に、データが複雑なほど「Multiple Stacking」や「Stacknet」の効果が上昇する  
- スタッキングモデルは上の階層に行くごとに、モデルを減らす必要がある。  
これは、オーバーフィッティングを防ぐためである


## 参考URL
- https://blog.ikedaosushi.com/entry/2018/12/15/212508
- https://qiita.com/r2en/items/61b3b7fa70d2b65021d0#heamy-multiple-stacking
### 「pystacknet」の参考ＵＲＬ
- https://github.com/h2oai/pystacknet/tree/af571e0b5517470563859a15eafb87b594e766eb


In [0]:
#環境設定


from heamy.dataset import Dataset
from heamy.estimator import Regressor
from heamy.estimator import Classifier
from heamy.pipeline import ModelsPipeline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error


In [0]:
#Single Stacking

seed=0

dataset = Dataset(X_train, y_train, X_test) # datasetの準備
models = [
    Classifier(dataset=dataset, estimator=RandomForestClassifier, parameters=(rfc_best_param), name='rf'),
    Classifier(dataset=dataset, estimator=ExtraTreesClassifier, parameters=(etc_best_param), name='et'),
    Classifier(dataset=dataset, estimator=XGBClassifier, parameters=(xgb_best_param), name='xg'),
    Classifier(dataset=dataset, estimator=LGBMClassifier, parameters=(lgb_best_param), name='lg')
]
#ステージ0の設定
# アンサンブルに使うモデルを定義
#「models」はスタッキングで利用する変数

pipeline = ModelsPipeline(*models)
stack_ds = pipeline.stack(k=10, seed=seed)
# pipelineを定義、ステージ1の準備


stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression)
y_trues, y_preds = stacker.validate(k=10)
#ステージ1の設定
#「estimator」は回帰分類は「LinearRegression(線形回帰)」
#クラス分けは「LogisticRegression(ロジスティック回帰)」
#ステージ1では、回帰問題においては線形回帰、分類問題においてはロジスティック回帰なども選択されますが、勾配ブースティングやランダムフォレストなどより複雑な学習器が使われることもあります。


train_sstacking=np.mean([accuracy_score(y_trues[ii],np.argmax(y_preds[ii],axis=1)) for ii in range(len(y_trues))])
#「train」モデルの正答率
pred=stacker.predict()
sstacking_result=accuracy_score(y_test,np.argmax(pred,axis=1))
#予想モデルの正答率

'''
# 精度出力
cv_results = []
for y_true, y_pred in zip(y_trues, y_preds):
    y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする
    cv_result = mean_absolute_error(y_true, y_pred_max)
    cv_results.append(cv_result)

1-np.mean(cv_results)
#「single stacking」の精度
#この結果はあくまで「TRAIN」データである
'''




In [0]:
#「weight average」

from sklearn.metrics import log_loss

# 最適な重みを探索
weights = pipeline.find_weights(log_loss)
pipeline_apply = pipeline.weight(weights)

cv_results = pipeline_apply.validate(scorer=log_loss, k=10)
train_wa=1-np.mean(cv_results)
#これが「train」データの正答率


y_pred = pipeline_apply.execute()
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする
weight_average_result=accuracy_score(y_pred_max,y_test)
#これが「test」データの正答率



In [0]:
# pipelineを定義、2nd levelデータセットの作成
pipeline2 = ModelsPipeline(*models)
stack_ds = pipeline2.stack(k=10, seed=seed)

# modelを作ってvalidation
#「estimator」は「LinearRegression(線形回帰)」でも「LogisticRegression(ロジスティック回帰)」でもどちらでもよい
stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression)
y_trues, y_preds = stacker.validate(k=10)



train_mstacking=np.mean([accuracy_score(y_trues[ii],np.argmax(y_preds[ii],axis=1)) for ii in range(len(y_trues))])
#「train」モデルの正答率
pred=stacker.predict()
mstacking_result=accuracy_score(y_test,np.argmax(pred,axis=1))
#「test」データの正答率

'''
# 精度出力
cv_results = []
for y_true, y_pred in zip(y_trues, y_preds):
    y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする
    cv_result = mean_absolute_error(y_true, y_pred_max)
    #cv_result = accuracy_score(y_true, y_pred_max)
    
    cv_results.append(cv_result)

1-np.mean(cv_results)
'''



#stacknet
1. 環境設定
>必要なライブラリをimport
2. 階層の準備
>level1とlevel2の変数を用意する
3. モデルの準備
4. 予測 
>「train」と「test」のそれぞれの予測正答率

In [0]:
#環境設定
from pystacknet.pystacknet import StackNetClassifier

In [0]:
#stacknetの階層を準備する
#もちろん、この階層をチューニングすることもできる
level1=[x[1] for x in estimators]
#「estimators」は「クラス分けモデル」の最後に設定した変数をそのまま利用している
level2=[RandomForestClassifier(**rfc_best_param)]
#二層目のモデルは簡単なものが一番だと思う
models=[level1,level2]


In [0]:
#モデルの準備
#今回はマルチクラス分けなので「logloss」を利用する
model=StackNetClassifier(models, metric="logloss", folds=5,
	restacking=False,use_retraining=True, use_proba=True, 
	random_state=12345,n_jobs=1, verbose=1)

#ここの「metric」は別で用意した関数を利用することができる
#なので「scikit-learn」の関数である、「metric=log_loss」のような使い方もできる

In [0]:
model.fit(X_train, y_train)
train_pred=model.predict_proba(X_train)
train_stacknet=accuracy_score(y_train,np.argmax(train_pred,axis=1))
#まずは「train」データの正答率



preds=model.predict_proba(X_test)
  
stacknet_result=accuracy_score( y_test,np.argmax(preds, axis=1))




In [0]:

sta_train_list=[train_sstacking,train_wa,train_mstacking,train_stacknet]
train_result.extend(sta_test_list)
#「train」データ

sta_test_list =[sstacking_result,weight_average_result,mstacking_result,stacknet_result]
RESULT.extend(sta_test_list)
#「test」データ

stack_names=['sstacking','wa','mstacking','stacknet']
model_names.extend(stack_names)
#model_names

In [0]:
res_df=pd.DataFrame([model_names,train_result,RESULT]).T
res_df.columns=['name','train','test']

In [0]:
res_df


# a

In [0]:
cmx=confusion_matrix(y_pred_max, y_test)

In [0]:
plot_confusion_matrix(model, X_test, y_test,
                      #display_labels=wine.target_names, 
                      cmap=plt.cm.Blues,)

In [0]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


cmd = ConfusionMatrixDisplay(cmx, y_test)
cmd.plot()
